### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from lightfm.datasets import fetch_movielens
from sklearn.metrics.pairwise import cosine_similarity
from collections import namedtuple

/Applications/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('RecSysHSE/ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
movie_info = pd.read_csv('RecSysHSE/ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [5]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [4]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [5]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [5]:
def to_csr(data, rows, cols):
    user_item = sp.coo_matrix((data, (rows, cols)))
    user_item_t_csr = user_item.T.tocsr()
    user_item_csr = user_item.tocsr()
    return user_item_csr, user_item_t_csr

#### Implicit data

In [6]:
users_imp = implicit_ratings["user_id"]
movies_imp = implicit_ratings["movie_id"]
user_item_imp, user_item_imp_t = to_csr(np.ones_like(users_imp), users_imp, movies_imp)

#### Explicit data

In [7]:
users_exp = ratings["user_id"]
movies_exp = ratings["movie_id"]
user_item_exp, user_item_exp_t = to_csr(ratings["rating"] , users_exp, movies_exp)

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [11]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [12]:
model.fit(user_item_imp_t)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [13]:
movie_info.head(5) 

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [15]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1526    Hercules (1997)',
 '1838    Mulan (1998)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [16]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [17]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [18]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_imp)]

In [19]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2502    Matrix, The (1999)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '3402    Close Encounters of the Third Kind (1977)',
 '1884    French Connection, The (1971)',
 '2880    Dr. No (1962)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Необходимые функции

In [8]:
def rmse(pred, real):
    nz = real.nonzero()
    pred = pred[nz].flatten()
    real = real[nz].flatten()
    pred = np.expand_dims(pred, axis=0)
    return mean_squared_error(real, pred, squared=False)

In [9]:
def get_initial_matrix(latent_size, return_bias=False):
    n_users, n_movies = user_item_imp.shape
    std = 1 / np.sqrt(latent_size)
    user = np.random.normal(scale=std, size=(n_users, latent_size))
    item = np.random.normal(scale=std, size=(n_movies, latent_size))
    
    if return_bias:
        b_u = np.random.normal(scale=std, size=(n_users))
        b_i = np.random.normal(scale=std,size=(n_movies))
        b_glob = np.random.normal(scale=std)
        return user, item, b_u, b_i, b_glob
        
    return user, item

In [10]:
def get_prediction(user, item):
    return user @ item.T

In [11]:
def title_by_id(id):
    return movie_info[movie_info['movie_id'] == id]['name'].to_string()

In [12]:
def get_similar_movies(movie_id, pred, k=10):
    print(f'For movie {title_by_id(movie_id)} similars are:')
    print()
    
    score = cosine_similarity(np.expand_dims(pred[movie_id], axis=0), pred)[0]
    ranked_similars = np.argsort(score)[::-1]
    top_k_ids = [s for s in ranked_similars if s != movie_id and s in movie_info['movie_id'].values][:k]
    top_k_titles = [title_by_id(r) for r in top_k_ids]
    print(*top_k_titles, sep='\n')

In [13]:
def predict_for_user(user_id, ratings, ratings_pred):
    real_movie_id = ratings[user_id].indices   
    print('--- User\'s choice ---')
    real_titles = [title_by_id(r) for r in real_movie_id[:10]]
    print(*real_titles, sep='\n')
    print()
    
    print('--- Our recommendations ---')
    user_pred = ratings_pred[user_id]
    ranked_movie_id = np.argsort(user_pred)[::-1]
    not_rated_movie_id = [i for i in ranked_movie_id if i not in real_movie_id and i in movie_info['movie_id'].values]
    pred_titles = [title_by_id(r) for r in not_rated_movie_id[:10]]
    print(*pred_titles, sep='\n')

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [15]:
def SVD(rating, user, item, b_u, b_i, b_glob, lambd, lr, epochs):
    non_zero_idx = list(zip(*rating.nonzero()))

    for k in tqdm(range(epochs)):
        np.random.shuffle(non_zero_idx)
        
        for idx in non_zero_idx:
            i, j = idx
            error = get_prediction(user[i], item[j]) + b_u[i] + b_i[j] + b_glob - rating[i, j] 
            user[i] -= lr * (error * item[j] + lambd * user[i])
            item[j] -= lr * (error * user[i] + lambd * item[j])
            b_u[i] -= lr * (error + lambd * b_u[i])
            b_i[j] -= lr * (error + lambd * b_i[j])
            b_glob -= lr * error
        
        if k % 5 == 0:
            loss = rmse(get_prediction(user, item), rating)
            print(f'{k}/{epochs} epoch: loss = {loss}')

    return user, item

In [20]:
W, H, b_u, b_i, b_glob = get_initial_matrix(latent_size=32, return_bias=True)
W, H = SVD(user_item_exp, W, H, b_u, b_i, b_glob, lambd=5e-3, lr=1e-2, epochs=40)

  2%|▎         | 1/40 [01:05<42:41, 65.67s/it]

0/40 epoch: loss = 3.581099606630683


 15%|█▌        | 6/40 [06:24<35:30, 62.66s/it]

5/40 epoch: loss = 3.5307313927647246


 28%|██▊       | 11/40 [11:46<31:22, 64.91s/it]

...

 78%|███████▊  | 31/40 [39:43<10:51, 72.40s/it]

30/40 epoch: loss = 3.43051750471871


 90%|█████████ | 36/40 [44:45<04:07, 61.87s/it]

35/40 epoch: loss = 3.427188101485979


100%|██████████| 40/40 [49:34<00:00, 74.36s/it]


#### похожие фильмы

In [21]:
movie_id = 1
get_similar_movies(movie_id, H)

For movie 0    Toy Story (1995) similars are:

3045    Toy Story 2 (1999)
2286    Bug's Life, A (1998)
584    Aladdin (1992)
591    Beauty and the Beast (1991)
2012    Little Mermaid, The (1989)
1838    Mulan (1998)
360    Lion King, The (1994)
2011    Lady and the Tramp (1955)
33    Babe (1995)
3682    Chicken Run (2000)


#### предсказания для пользователя

In [22]:
user_id = 4
predict_for_user(user_id, user_item_imp, get_prediction(W, H))

--- User's choice ---
257    Star Wars: Episode IV - A New Hope (1977)
476    Jurassic Park (1993)
1023    Die Hard (1988)
1081    E.T. the Extra-Terrestrial (1982)
1180    Raiders of the Lost Ark (1981)
1183    Good, The Bad and The Ugly, The (1966)
1196    Alien (1979)
1220    Terminator, The (1984)
1366    Jaws (1975)
1885    Rocky (1976)

--- Our recommendations ---
2102    Next Stop, Wonderland (1998)
2249    Happiness (1998)
3673    Battleship Potemkin, The (Bronenosets Potyomki...
2506    Dreamlife of Angels, The (La Vie r�v�e des ang...
502    Orlando (1993)
1553    In the Company of Men (1997)
3847    Hellraiser (1987)
1745    Hana-bi (1997)
377    When a Man Loves a Woman (1994)
1507    Schizopolis (1996)


### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [57]:
def ALS(rating, user, item, epochs, lambd):
    for i in tqdm(range(epochs)):
        user = als_step(item, rating, lambd)
        item = als_step(user, rating.T, lambd)
    
        if i % 1000 == 0:
            loss = rmse(get_prediction(user, item), rating)
            print(f'{i}/{epochs} epoch: loss = {loss}')

    return user, item
    
def als_step(fixed, rating, lambd):
    X = fixed.T @ fixed + lambd * np.eye(fixed.shape[1])
    X = np.linalg.inv(X)
    X = fixed @ X.T
    return rating @ X

In [58]:
U, V = get_initial_matrix(latent_size=32)
U, V = ALS(user_item_imp, U, V, epochs=3000, lambd=0.01)

  0%|          | 5/3000 [00:00<07:17,  6.85it/s]

0/3000 epoch: loss = 0.7568758437446995


 34%|███▎      | 1007/3000 [00:31<01:14, 26.61it/s]

1000/3000 epoch: loss = 0.627284696786587


 67%|██████▋   | 2007/3000 [01:01<00:36, 26.96it/s]

2000/3000 epoch: loss = 0.6272372994997109


100%|██████████| 3000/3000 [01:32<00:00, 32.55it/s]


#### похожие фильмы

In [59]:
movie_id = 1
get_similar_movies(movie_id, V)

For movie 0    Toy Story (1995) similars are:

3045    Toy Story 2 (1999)
33    Babe (1995)
584    Aladdin (1992)
2286    Bug's Life, A (1998)
1245    Groundhog Day (1993)
2252    Pleasantville (1998)
360    Lion King, The (1994)
591    Beauty and the Beast (1991)
2315    Babe: Pig in the City (1998)
2225    Antz (1998)


#### предсказания для пользователя

In [60]:
user_id = 4
predict_for_user(user_id, user_item_imp, get_prediction(U, V))

--- User's choice ---
257    Star Wars: Episode IV - A New Hope (1977)
476    Jurassic Park (1993)
1023    Die Hard (1988)
1081    E.T. the Extra-Terrestrial (1982)
1180    Raiders of the Lost Ark (1981)
1183    Good, The Bad and The Ugly, The (1966)
1196    Alien (1979)
1220    Terminator, The (1984)
1366    Jaws (1975)
1885    Rocky (1976)

--- Our recommendations ---
1178    Star Wars: Episode V - The Empire Strikes Back...
585    Terminator 2: Judgment Day (1991)
1271    Indiana Jones and the Last Crusade (1989)
1182    Aliens (1986)
847    Godfather, The (1972)
2502    Matrix, The (1999)
1192    Star Wars: Episode VI - Return of the Jedi (1983)
1284    Butch Cassidy and the Sundance Kid (1969)
453    Fugitive, The (1993)
1203    Godfather: Part II, The (1974)


### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [93]:
# ### OLD

# def BPR(D, user, item, iters, lr, lambd):  
#     for k in tqdm(range(iters)):
#         u = np.random.choice(user_ids)

#         while not len(D[u].pos):
#             u = np.random.choice(user_ids)
            
#         i = np.random.choice(D[u].pos)
#         j = np.random.choice(D[u].neg)
#         x_uij_hat = get_prediction(user[u], item[i]) - get_prediction(user[u], item[j])
        
#         user[u] += lr * (bpr_step(x_uij_hat, item[i] - item[j]) + lambd * user[u])
#         item[i] += lr * (bpr_step(x_uij_hat, user[u]) + lambd * item[i])
#         item[j] += lr * (bpr_step(x_uij_hat, -user[u]) + lambd * item[j]) 
#     return user, item

# def bpr_step(x_uij_hat, partial_d):
#     exp_x = np.exp(-x_uij_hat)
#     return exp_x / (1 + exp_x) * partial_d

In [23]:
user_ids = np.unique(ratings['user_id'].values)

In [24]:
def pos_neg_dataset(csr_ratings):
    movie_ids = movie_info['movie_id'].values
    User = namedtuple('User', ['pos', 'neg'])
    Ds = dict()
    
    for u in user_ids:
        user_rating = csr_ratings[u]
        rated = list(user_rating.indices)
        not_rated = list(np.setdiff1d(movie_ids, rated))
        user = User(rated, not_rated)
        Ds[u] = user
    return Ds

In [25]:
Ds = pos_neg_dataset(user_item_imp)

In [37]:
def BPR(D, user, item, iters, lr, lambd):  
    for k in tqdm(range(iters)):
        np.random.shuffle(user_ids)
        
        for u in user_ids:
            np.random.shuffle(D[u].pos)
            
            for i in D[u].pos:
                for _ in range(5):
                    j = np.random.choice(D[u].neg)

                    x_uij_hat = get_prediction(user[u], item[i]) - get_prediction(user[u], item[j])
                    user[u] += lr * (bpr_step(x_uij_hat, item[i] - item[j]) + lambd * user[u])
                    item[i] += lr * (bpr_step(x_uij_hat, user[u]) + lambd * item[i])
                    item[j] += lr * (bpr_step(x_uij_hat, -user[u]) + lambd * item[j]) 
    return user, item

def bpr_step(x_uij_hat, partial_d):
    exp_x = np.exp(-x_uij_hat)
    return exp_x / (1 + exp_x) * partial_d

In [38]:
U, V = get_initial_matrix(latent_size=100)
U, V = BPR(Ds, U, V, iters=5, lr=3e-2, lambd=1e-5)

100%|██████████| 5/5 [1:30:36<00:00, 1087.29s/it]


#### похожие фильмы

In [39]:
movie_id = 1
get_similar_movies(movie_id, V)

For movie 0    Toy Story (1995) similars are:

584    Aladdin (1992)
3045    Toy Story 2 (1999)
2286    Bug's Life, A (1998)
591    Beauty and the Beast (1991)
1854    There's Something About Mary (1998)
360    Lion King, The (1994)
2252    Pleasantville (1998)
33    Babe (1995)
352    Forrest Gump (1994)
1245    Groundhog Day (1993)


#### предсказания для пользователя

In [40]:
user_id = 4
predict_for_user(user_id, user_item_imp, get_prediction(U, V))

--- User's choice ---
257    Star Wars: Episode IV - A New Hope (1977)
476    Jurassic Park (1993)
1023    Die Hard (1988)
1081    E.T. the Extra-Terrestrial (1982)
1180    Raiders of the Lost Ark (1981)
1183    Good, The Bad and The Ugly, The (1966)
1196    Alien (1979)
1220    Terminator, The (1984)
1366    Jaws (1975)
1885    Rocky (1976)

--- Our recommendations ---
1178    Star Wars: Episode V - The Empire Strikes Back...
585    Terminator 2: Judgment Day (1991)
2502    Matrix, The (1999)
537    Blade Runner (1982)
1182    Aliens (1986)
1192    Star Wars: Episode VI - Return of the Jedi (1983)
1353    Star Trek: The Wrath of Khan (1982)
1284    Butch Cassidy and the Sundance Kid (1969)
847    Godfather, The (1972)
1204    Full Metal Jacket (1987)


### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [45]:
def WARP(D, user, item, iters, lr):
    for k in tqdm(range(iters)):
        np.random.shuffle(user_ids)
        
        for u in user_ids:
            np.random.shuffle(D[u].pos)
            
            for i in D[u].pos:
                negatives = D[u].neg
                np.random.shuffle(negatives)

                pred = get_prediction(user[u], item)
                pos_score = pred[i]

                for n_neg, j in enumerate(negatives):
                    neg_score = pred[j]

                    if neg_score > pos_score - 1:
                        n_neg += 1
                        rank = len(negatives) / n_neg
                        user[u] -= lr * np.log(rank) * (-item[i] + item[j])
                        item[i] -= lr * np.log(rank) * (-user[u])
                        item[j] -= lr * np.log(rank) * (user[u])
                        break
    return user, item

In [46]:
U, V = get_initial_matrix(latent_size=64)
U, V = WARP(Ds, U, V, iters=20, lr=1e-3)

100%|██████████| 20/20 [1:28:46<00:00, 266.34s/it]


#### похожие фильмы

In [49]:
movie_id = 1
get_similar_movies(movie_id, V)

For movie 0    Toy Story (1995) similars are:

3045    Toy Story 2 (1999)
584    Aladdin (1992)
2286    Bug's Life, A (1998)
1029    That Thing You Do! (1996)
2692    Iron Giant, The (1999)
2012    Little Mermaid, The (1989)
1526    Hercules (1997)
2315    Babe: Pig in the City (1998)
1838    Mulan (1998)
510    Ref, The (1994)


#### предсказания для пользователя

In [50]:
user_id = 4
predict_for_user(user_id, user_item_imp, get_prediction(U, V))

--- User's choice ---
257    Star Wars: Episode IV - A New Hope (1977)
476    Jurassic Park (1993)
1023    Die Hard (1988)
1081    E.T. the Extra-Terrestrial (1982)
1180    Raiders of the Lost Ark (1981)
1183    Good, The Bad and The Ugly, The (1966)
1196    Alien (1979)
1220    Terminator, The (1984)
1366    Jaws (1975)
1885    Rocky (1976)

--- Our recommendations ---
3458    Predator (1987)
108    Braveheart (1995)
1178    Star Wars: Episode V - The Empire Strikes Back...
2502    Matrix, The (1999)
1539    Men in Black (1997)
847    Godfather, The (1972)
1271    Indiana Jones and the Last Crusade (1989)
957    African Queen, The (1951)
2460    Planet of the Apes (1968)
585    Terminator 2: Judgment Day (1991)
